# ブロックチェ－ンを構築しながら学ぶ（Jupyter版）

* [ブロックチェ－ンを構築しながら学ぶ | プログラミング | POSTD](http://postd.cc/learn-blockchains-by-building-one/) で公開されていた解説とコードを使って step-by-step で動かしてみるテストです。
* Flaskでの実装は本質的でなく冗長であると感じたので、Jupyterで実装してます。
* ブロックチェーンのしくみそのものについては、元記事の解説を読んで理解することを前提としています。

## コードの実装
### まずは Blockchain クラスを作成

In [1]:
from time import time
import hashlib
import json

class Blockchain(object):
    def __init__(self):
        self.current_transactions = []
        self.chain = []

        # Create the genesis block
        self.new_block(previous_hash=1, proof=100)

    def new_block(self, proof, previous_hash=None):
        """
        Create a new Block in the Blockchain
        :param proof: <int> The proof given by the Proof of Work algorithm
        :param previous_hash: (Optional) <str> Hash of previous Block
        :return: <dict> New Block
        """
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }

        # Reset the current list of transactions
        self.current_transactions = []

        self.chain.append(block)
        return block

    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block
        :param sender: <str> Address of the Sender
        :param recipient: <str> Address of the Recipient
        :param amount: <int> Amount
        :return: <int> The index of the Block that will hold this transaction
        """
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })
        return self.last_block['index'] + 1

    @staticmethod
    def hash(block):
        """
        Creates a SHA-256 hash of a Block
        :param block: <dict> Block
        :return: <str>
        """
        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    @property
    def last_block(self):
        return self.chain[-1]

    def proof_of_work(self, last_proof):
        """
        Simple Proof of Work Algorithm:
         - Find a number p' such that hash(pp') contains leading 4 zeroes, where p is the previous p'
         - p is the previous proof, and p' is the new proof
        :param last_proof: <int>
        :return: <int>
        """
        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
        return proof

    @staticmethod
    def valid_proof(last_proof, proof):
        """
        Validates the Proof: Does hash(last_proof, proof) contain 4 leading zeroes?
        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :return: <bool> True if correct, False if not.
        """
        guess = ('%d%d' % (last_proof, proof)).encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"


### 次にマイニングのルーチンを追加

In [2]:
from uuid import uuid4

node_identifier = str(uuid4()).replace('-', '')

def mine(blockchain):
    global node_identifier
    
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0",
        recipient=node_identifier,
        amount=1,
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
    }
    return response

### 最後にチェーン全体を取得するルーチンを追加

In [3]:
def full_chain(blockchain):
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return response

# ブロックチェーンを動かしてみる

表示を見やすくするために pprint を使う。

In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

自ノードを示す識別子は以下。

In [5]:
node_identifier

'33f7be1eacda482b9b45891ce02a8adc'

インスタンス化すると、最初のブロックが作成される。
最初のブロックは
* インデックス1
* 前のハッシュは無し（1）
* チェーンの長さは当然ながら「1」

In [6]:
b = Blockchain()
pp.pprint(full_chain(b))

{ 'chain': [ { 'index': 1,
               'previous_hash': 1,
               'proof': 100,
               'timestamp': 1514434182.5600014,
               'transactions': []}],
  'length': 1}


ひとつブロックをマイニングしてみる。

In [7]:
newblock = mine(b)
pp.pprint(newblock)

{ 'index': 2,
  'message': 'New Block Forged',
  'previous_hash': '4b33282ad2523d8f042d2df4d3fa917ac4069724bb273a107deb9c122ff92eec',
  'proof': 35293,
  'transactions': [ { 'amount': 1,
                      'recipient': '33f7be1eacda482b9b45891ce02a8adc',
                      'sender': '0'}]}


transactions には、マイニングしたというトランザクションのみが記載されている。
マイニングのトランザクションは

* sender が 0
* recipient が自ノードの識別子
* amount が 1

のトランザクションとして記録されている。

この時のブロックチェーン全体の内容を見てみる。

In [8]:
pp.pprint(full_chain(b))

{ 'chain': [ { 'index': 1,
               'previous_hash': 1,
               'proof': 100,
               'timestamp': 1514434182.5600014,
               'transactions': []},
             { 'index': 2,
               'previous_hash': '4b33282ad2523d8f042d2df4d3fa917ac4069724bb273a107deb9c122ff92eec',
               'proof': 35293,
               'timestamp': 1514434182.7299504,
               'transactions': [ { 'amount': 1,
                                   'recipient': '33f7be1eacda482b9b45891ce02a8adc',
                                   'sender': '0'}]}],
  'length': 2}


新しい取引（トランザクション）としては、マイニング結果だけを含むブロックが存在している。

ここに新しいトランザクションを追加してみる。
* sender を 'foo'
* recipient を 'bar'
* amount を 10

のトランザクションとする。

In [9]:
index = b.new_transaction('foo', 'bar', 10)

この時の（ブロックの）インデックスは3になる。このブロックに上記のトランザクションは保存される。

In [10]:
print(index)

3


この時、チェーン全体を見てみると、上記で追加したトランザクションはまだチェーンに登録されていない。

In [11]:
pp.pprint(full_chain(b))

{ 'chain': [ { 'index': 1,
               'previous_hash': 1,
               'proof': 100,
               'timestamp': 1514434182.5600014,
               'transactions': []},
             { 'index': 2,
               'previous_hash': '4b33282ad2523d8f042d2df4d3fa917ac4069724bb273a107deb9c122ff92eec',
               'proof': 35293,
               'timestamp': 1514434182.7299504,
               'transactions': [ { 'amount': 1,
                                   'recipient': '33f7be1eacda482b9b45891ce02a8adc',
                                   'sender': '0'}]}],
  'length': 2}


マイニングして新しいブロックを追加する。

In [12]:
newblock = mine(b)

3番目のブロックが作成され、先に作成したトランザクションの情報とマイニングの情報が保存される。

In [13]:
pp.pprint(newblock)

{ 'index': 3,
  'message': 'New Block Forged',
  'previous_hash': 'c812d75ea69333c77c77ee2f251bbf478bd29657c88c1e1f0b3f84667e289222',
  'proof': 35089,
  'transactions': [ {'amount': 10, 'recipient': 'bar', 'sender': 'foo'},
                    { 'amount': 1,
                      'recipient': '33f7be1eacda482b9b45891ce02a8adc',
                      'sender': '0'}]}


この時のチェーン全体の状態は以下。

In [14]:
pp.pprint(full_chain(b))

{ 'chain': [ { 'index': 1,
               'previous_hash': 1,
               'proof': 100,
               'timestamp': 1514434182.5600014,
               'transactions': []},
             { 'index': 2,
               'previous_hash': '4b33282ad2523d8f042d2df4d3fa917ac4069724bb273a107deb9c122ff92eec',
               'proof': 35293,
               'timestamp': 1514434182.7299504,
               'transactions': [ { 'amount': 1,
                                   'recipient': '33f7be1eacda482b9b45891ce02a8adc',
                                   'sender': '0'}]},
             { 'index': 3,
               'previous_hash': 'c812d75ea69333c77c77ee2f251bbf478bd29657c88c1e1f0b3f84667e289222',
               'proof': 35089,
               'timestamp': 1514434183.0127025,
               'transactions': [ { 'amount': 10,
                                   'recipient': 'bar',
                                   'sender': 'foo'},
                                 { 'amount': 1,
                       

## 分散合意

### Blockchain2 クラスの実装

コンセンサスアルゴリズムを組み込んだ Blcokchain2 クラスを実装する。

ついでに、ノード識別子をクラスメンバとして保持し、マイニングした時にはそれを使うように修正。（複数のノードのブロックチェーンを扱えるようにするため）

（本当は Node クラスを定義して has-a として Blockchain2 クラスをメンバに持った方がいい気がするが、オリジナル版で register_node() とか resolve_conflicts() とかが Blockchain クラスに導入されているので、それを踏襲する。）

In [15]:
import copy

BlockchainNeighbours = {}

class Blockchain2(Blockchain):
    def __init__(self, node_identifier):
        super().__init__()
        self.nodes = set()
        self.node_identifier = node_identifier
        
    def register_node(self, node_identifier):
        """
        Add a new node to the list of nodes
        :node_identifier: <str> Node identifier of the neighbour node.
        :return: None
        """
        self.nodes.add(node_identifier)

    def valid_chain(self, chain):
        """
        Determine if a given blockchain is valid
        :param chain: <list> A blockchain
        :return: <bool> True if valid, False if not
        """

        last_block = chain[0]
        current_index = 1

        while current_index < len(chain):
            block = chain[current_index]
#            print(f'{last_block}')
#            print(f'{block}')
#            print("\n-----------\n")
            # Check that the hash of the block is correct
            if block['previous_hash'] != self.hash(last_block):
                return False

            # Check that the Proof of Work is correct
            if not self.valid_proof(last_block['proof'], block['proof']):
                return False

            last_block = block
            current_index += 1

        return True

    def resolve_conflicts(self):
        """
        This is our Consensus Algorithm, it resolves conflicts
        by replacing our chain with the longest one in the network.
        :return: <bool> True if our chain was replaced, False if not
        """
        neighbours = self.nodes
        new_chain = None

        # We're only looking for chains longer than ours
        max_length = len(self.chain)

        # Grab and verify the chains from all the nodes in our network
        for node in neighbours:
            blockchain = BlockchainNeighbours[node]
            print('node id: %s, len: %d' % (blockchain.node_identifier, len(blockchain.chain)))

            # Check if the length is longer and the chain is valid
            if len(blockchain.chain) > max_length and self.valid_chain(blockchain.chain):
                max_length = len(blockchain.chain)
                new_chain = blockchain

        # Replace our chain if we discovered a new, valid chain longer than ours
        if new_chain:
            print("Replacing `%s' <- `%s'" % (self.node_identifier, new_chain.node_identifier))
            self.chain = copy.copy(new_chain.chain)
            return True

        return False

In [16]:
def mine2(blockchain):
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0",
        recipient=blockchain.node_identifier,
        amount=1,
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
    }
    return response

### 複数ノードのブロックチェーンの作成

3ノード分のブロックチェーンを作成して、近隣ノードとして登録する。

In [17]:
# ノード識別子を 'foo', 'bar', 'buz' として3ノード分のブロックチェーンを作成する。
foo = Blockchain2('foo')
bar = Blockchain2('bar')
buz = Blockchain2('buz')

# 近隣ノードの一覧に登録
BlockchainNeighbours['foo'] = foo
BlockchainNeighbours['bar'] = bar
BlockchainNeighbours['buz'] = buz

# 'foo' ノードの近隣として 'bar', 'buz' を登録
foo.register_node('bar')
foo.register_node('buz')

# 'bar', 'buz' ノードについても近隣を登録
bar.register_node('foo')
bar.register_node('buz')

buz.register_node('foo')
buz.register_node('bar')

初期状態だと、すべてのノードのチェーンは長さが1となる。

In [18]:
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))

foo: 1, bar: 1, buz: 1


初期状態でコンフリクトを解消しようとしても、fooノードより長いチェーンを持つノードが存在しないので、fooノードのチェーンは変更されない。

In [19]:
foo.resolve_conflicts()

node id: buz, len: 1
node id: bar, len: 1


False

### 一部のノードにおけるブロックの伸長

次に bar ノードでマイニングし、ブロックを1つ追加する。

In [20]:
pp.pprint(mine2(bar))

{ 'index': 2,
  'message': 'New Block Forged',
  'previous_hash': '6d2219c4cfd1bc5c3531f737447bb50327e2474fe67a525abbc288a6c60da977',
  'proof': 35293,
  'transactions': [{'amount': 1, 'recipient': 'bar', 'sender': '0'}]}


すると、barノードだけ長さが2となる。

In [21]:
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))

foo: 1, bar: 2, buz: 1


In [22]:
pp.pprint(foo.chain)

[ { 'index': 1,
    'previous_hash': 1,
    'proof': 100,
    'timestamp': 1514434183.2797077,
    'transactions': []}]


In [23]:
pp.pprint(bar.chain)

[ { 'index': 1,
    'previous_hash': 1,
    'proof': 100,
    'timestamp': 1514434183.2797077,
    'transactions': []},
  { 'index': 2,
    'previous_hash': '6d2219c4cfd1bc5c3531f737447bb50327e2474fe67a525abbc288a6c60da977',
    'proof': 35293,
    'timestamp': 1514434183.493838,
    'transactions': [{'amount': 1, 'recipient': 'bar', 'sender': '0'}]}]


In [24]:
pp.pprint(buz.chain)

[ { 'index': 1,
    'previous_hash': 1,
    'proof': 100,
    'timestamp': 1514434183.2797077,
    'transactions': []}]


### ノード間のコンフリクトの解消

この状態で foo ノードでコンフリクトを解消しようとすると、fooノードのチェーンは（より長い）barノードのチェーンで上書きされる。

In [25]:
foo.resolve_conflicts()

node id: buz, len: 1
node id: bar, len: 2
Replacing `foo' <- `bar'


True

コンフリクトの解消が完了すると、fooノードのチェーンの長さが2となる。

In [26]:
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))

foo: 2, bar: 2, buz: 1


### ノードごとにチェーンの内容が異なる場合

次に、ノードごとにチェーンの内容が異なるケースを考えてみる。

ここでは、fooノードとbuzノードの内容が異なる場合を見る。

In [27]:
# buzノードの内容を揃える
buz.resolve_conflicts()
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))

node id: foo, len: 2
node id: bar, len: 2
Replacing `buz' <- `foo'
foo: 2, bar: 2, buz: 2


ここで、fooノードで2ブロック、buzノードで1ブロック、それぞれ異なるトランザクションを保持しているブロックを追加する。

In [28]:
foo.new_transaction('AAA', 'BBB', 123)
mine2(foo)
foo.new_transaction('CCC', 'DDD', 456)
mine2(foo)

buz.new_transaction('EEE', 'FFF', 789)
mine2(buz)

print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))

foo: 4, bar: 2, buz: 3


この時の foo ノードと buz ノードのチェーンの内容は以下の通り。途中から内容が異なっていることが分かる。

In [29]:
pp.pprint(foo.chain)

[ { 'index': 1,
    'previous_hash': 1,
    'proof': 100,
    'timestamp': 1514434183.2797077,
    'transactions': []},
  { 'index': 2,
    'previous_hash': '6d2219c4cfd1bc5c3531f737447bb50327e2474fe67a525abbc288a6c60da977',
    'proof': 35293,
    'timestamp': 1514434183.493838,
    'transactions': [{'amount': 1, 'recipient': 'bar', 'sender': '0'}]},
  { 'index': 3,
    'previous_hash': '954623f55a83006026fbfd26b1bff1992bacc4cdd025db25a5f4adf2f8842725',
    'proof': 35089,
    'timestamp': 1514434183.8250325,
    'transactions': [ {'amount': 123, 'recipient': 'BBB', 'sender': 'AAA'},
                      {'amount': 1, 'recipient': 'foo', 'sender': '0'}]},
  { 'index': 4,
    'previous_hash': 'ea8803b7328c94682b02c9fd539ecfa873d529551413047f8f1e50bb95003dd8',
    'proof': 119678,
    'timestamp': 1514434184.1612155,
    'transactions': [ {'amount': 456, 'recipient': 'DDD', 'sender': 'CCC'},
                      {'amount': 1, 'recipient': 'foo', 'sender': '0'}]}]


In [30]:
pp.pprint(buz.chain)

[ { 'index': 1,
    'previous_hash': 1,
    'proof': 100,
    'timestamp': 1514434183.2797077,
    'transactions': []},
  { 'index': 2,
    'previous_hash': '6d2219c4cfd1bc5c3531f737447bb50327e2474fe67a525abbc288a6c60da977',
    'proof': 35293,
    'timestamp': 1514434183.493838,
    'transactions': [{'amount': 1, 'recipient': 'bar', 'sender': '0'}]},
  { 'index': 3,
    'previous_hash': '954623f55a83006026fbfd26b1bff1992bacc4cdd025db25a5f4adf2f8842725',
    'proof': 35089,
    'timestamp': 1514434184.251345,
    'transactions': [ {'amount': 789, 'recipient': 'FFF', 'sender': 'EEE'},
                      {'amount': 1, 'recipient': 'buz', 'sender': '0'}]}]


この状態で foo ノードでコンフリクトを解消しようとしてもチェーンの内容は変わらない。

In [31]:
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))
print(foo.resolve_conflicts())
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))

foo: 4, bar: 2, buz: 3
node id: buz, len: 3
node id: bar, len: 2
False
foo: 4, bar: 2, buz: 3


一方で buz ノードでコンフリクトを解消すると、チェーンの内容が書き換わる。

In [32]:
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))
print(buz.resolve_conflicts())
print('foo: %d, bar: %d, buz: %d' %(len(foo.chain), len(bar.chain), len(buz.chain)))

foo: 4, bar: 2, buz: 3
node id: foo, len: 4
node id: bar, len: 2
Replacing `buz' <- `foo'
True
foo: 4, bar: 2, buz: 4


結果的に、buzノードのチェーンは破棄され、fooノードのチェーンで上書きされる。

In [33]:
pp.pprint(foo.chain)

[ { 'index': 1,
    'previous_hash': 1,
    'proof': 100,
    'timestamp': 1514434183.2797077,
    'transactions': []},
  { 'index': 2,
    'previous_hash': '6d2219c4cfd1bc5c3531f737447bb50327e2474fe67a525abbc288a6c60da977',
    'proof': 35293,
    'timestamp': 1514434183.493838,
    'transactions': [{'amount': 1, 'recipient': 'bar', 'sender': '0'}]},
  { 'index': 3,
    'previous_hash': '954623f55a83006026fbfd26b1bff1992bacc4cdd025db25a5f4adf2f8842725',
    'proof': 35089,
    'timestamp': 1514434183.8250325,
    'transactions': [ {'amount': 123, 'recipient': 'BBB', 'sender': 'AAA'},
                      {'amount': 1, 'recipient': 'foo', 'sender': '0'}]},
  { 'index': 4,
    'previous_hash': 'ea8803b7328c94682b02c9fd539ecfa873d529551413047f8f1e50bb95003dd8',
    'proof': 119678,
    'timestamp': 1514434184.1612155,
    'transactions': [ {'amount': 456, 'recipient': 'DDD', 'sender': 'CCC'},
                      {'amount': 1, 'recipient': 'foo', 'sender': '0'}]}]


In [34]:
pp.pprint(buz.chain)

[ { 'index': 1,
    'previous_hash': 1,
    'proof': 100,
    'timestamp': 1514434183.2797077,
    'transactions': []},
  { 'index': 2,
    'previous_hash': '6d2219c4cfd1bc5c3531f737447bb50327e2474fe67a525abbc288a6c60da977',
    'proof': 35293,
    'timestamp': 1514434183.493838,
    'transactions': [{'amount': 1, 'recipient': 'bar', 'sender': '0'}]},
  { 'index': 3,
    'previous_hash': '954623f55a83006026fbfd26b1bff1992bacc4cdd025db25a5f4adf2f8842725',
    'proof': 35089,
    'timestamp': 1514434183.8250325,
    'transactions': [ {'amount': 123, 'recipient': 'BBB', 'sender': 'AAA'},
                      {'amount': 1, 'recipient': 'foo', 'sender': '0'}]},
  { 'index': 4,
    'previous_hash': 'ea8803b7328c94682b02c9fd539ecfa873d529551413047f8f1e50bb95003dd8',
    'proof': 119678,
    'timestamp': 1514434184.1612155,
    'transactions': [ {'amount': 456, 'recipient': 'DDD', 'sender': 'CCC'},
                      {'amount': 1, 'recipient': 'foo', 'sender': '0'}]}]


以上。